<a href="https://colab.research.google.com/github/beatakovacs/Hogwarts-Express/blob/master/hogwarts_express.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My Drive/HogwartsExpress'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
repo_url = 'https://github.com/beatakovacs/Hogwarts-Express'
num_steps = 50000
selected_model = 'ssd_mobilenet_v2_quantized_coco'
MODEL = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03'
pipeline_file = 'ssd_mobilenet_v2_quantized_300x300_coco.config'

In [11]:
import os

%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
fatal: destination path 'Hogwarts-Express' already exists and is not an empty directory.
/content/Hogwarts-Express
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/beatakovacs/Hogwarts-Express
   00c07cf..ee71610  master     -> origin/master
Updating 00c07cf..ee71610
Fast-forward
 frozen_inference_graph.pb | Bin 0 -> 19734797 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 frozen_inference_graph.pb


In [3]:

%cd /content
!git clone https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

/content
Cloning into 'Hogwarts-Express'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 2476 (delta 2), reused 13 (delta 2), pack-reused 2463
Receiving objects: 100% (2476/2476), 368.17 MiB | 37.19 MiB/s, done.
Resolving deltas: 100% (1219/1219), done.
Checking out files: 100% (2470/2470), done.
/content/Hogwarts-Express
Already up to date.
/content
Cloning into 'models'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 33183 (delta 0), reused 7 (delta 0), pack-reused 33176
Receiving objects: 100% (33183/33183), 511.88 MiB | 36.22 MiB/s, done.
Resolving deltas: 100% (21169/21169), done.
Checking out files: 100% (3188/3188), done.
Selecting previously unselected package python-bs4.
(Reading database ... 145653 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_

In [0]:
test_record_fname = repo_dir_path + '/annotations/test.record'
train_record_fname = repo_dir_path + '/annotations/train.record'
label_map_pbtxt_fname = repo_dir_path + '/annotations/label_map.pbtxt'

In [5]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [0]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")

In [0]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

In [0]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [9]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [0]:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir='{model_dir}' \
      --alsologtostderr \
      --num_train_steps={num_steps}